In [11]:
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

print(os.getcwd())
print(os.getcwd())
os.chdir('/home/jelinek/recetox/')

from ml.eval import eval_model
from ml.pipeline import FeitDataPipeline
from tensorflow import keras

from tensorflow.keras.callbacks import ReduceLROnPlateau

from cfg import *

from livelossplot import PlotLossesKerasTF


name = "MySimpleCNN-v2-reduced-atrous-convolution"


/home/jelinek/recetox


In [12]:
from ml.util import FeitClasMapGen
from keras import activations


class DeepLabV3(FeitDataPipeline):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.model = self.get_compiled_model()
        self.params.name = name
        self.params.epochs = 200
        self.batch_size = 16
        self.params.tile_size=256
        self.params.number_of_classes = 12

        self.data_loader_training = self.get_data_loader_training()
        self.data_loader_validation = self.get_data_loader_validation()

    @staticmethod
    def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
        x = keras.layers.Conv2D(num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding="same",
            use_bias=use_bias, kernel_initializer=keras.initializers.HeNormal())(block_input)
        """
        Function source: https://keras.io/examples/vision/deeplabv3_plus/
        """
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activations.relu)(x)
        return x

    @staticmethod
    def DilatedSpatialPyramidPooling(dspp_input):
        """
        Function source: https://keras.io/examples/vision/deeplabv3_plus/
        """

        dims = dspp_input.shape
        x = keras.layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
        x = DeepLabV3.convolution_block(x, kernel_size=1, use_bias=True)
        out_pool = keras.layers.UpSampling2D(
            size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",)(x)

        out_1 = DeepLabV3.convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
        out_6 = DeepLabV3.convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
        out_12 = DeepLabV3.convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
        out_18 = DeepLabV3.convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

        x = keras.layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
        output = DeepLabV3.convolution_block(x, kernel_size=1)
        return output


    @staticmethod
    def get_compiled_model():
        image_size = 256
        model_input = keras.Input(shape=(256, 256, 3))
        resnet50 = keras.applications.ResNet50(
            weights="imagenet", include_top=False, input_tensor=model_input
        )
        x = resnet50.get_layer("conv4_block6_2_relu").output
        x = DeepLabV3.DilatedSpatialPyramidPooling(x)
    
        input_a = keras.layers.UpSampling2D(
            size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
            interpolation="bilinear",
        )(x)
        input_b = resnet50.get_layer("conv2_block3_2_relu").output
        input_b = DeepLabV3.convolution_block(input_b, num_filters=48, kernel_size=1)
    
        x = keras.layers.Concatenate(axis=-1)([input_a, input_b])
        x = DeepLabV3.convolution_block(x)
        x = DeepLabV3.convolution_block(x)
        x = keras.layers.UpSampling2D(
                    size=(image_size // x.shape[1], image_size // x.shape[2]),
                    interpolation="bilinear",)(x)
        model_output = keras.layers.Conv2D(11, kernel_size=(1, 1), padding="same")(x)
        model =  keras.Model(inputs=model_input, outputs=model_output)

        # model.summary()
        return model


    def execute_pipeline(self, perform_validation=True, save_model=True, perform_test_segmentation=True):
        data_train = self.data_loader_training
        data_valid = self.data_loader_validation

        self.model.compile(loss='sparse_categorical_crossentropy',
                           optimizer=self.get_optimizer(),
                           metrics=['accuracy'])


        self._train_model(data_train, data_valid)

    def get_optimizer(self):
        return keras.optimizers.Adam(learning_rate=1e-4, clipvalue=1.)

    def get_data_loader_training(self):
        datagen_train = FeitClasMapGen(horizontal_flip=False, vertical_flip=False, samplewise_center=False,
                                           samplewise_std_normalization=True)

        return datagen_train.flow_from_directory(directory=self.params.data_training, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size, self.params.tile_size),
                                                 broadcast=True)

    def get_data_loader_validation(self):
        datagen_valid = FeitClasMapGen(samplewise_center=False, samplewise_std_normalization=True)
        return datagen_valid.flow_from_directory(directory=self.params.data_validation, color_mode='rgb',
                                                 class_mode='categorical', batch_size=self.params.batch_size,
                                                 shuffle=True,
                                                 target_size=(self.params.tile_size, self.params.tile_size),
                                                 broadcast=True)


    def _train_model(self, data_train, data_valid):

        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                      patience=30, min_lr=1e-4, verbose=1,
                                      cooldown=20)

        class_weights = {k : 1.0 for k in range(self.params.number_of_classes)}
        class_weights[self.params.number_of_classes] = 0.0

        self.model.fit(data_train,
                       steps_per_epoch=250,
                       epochs=20,
                       shuffle=True,
                       validation_data=data_valid,
                       validation_freq=100,
                       verbose=1,
                       validation_steps=10000,
                       callbacks=[self.tensorboard, reduce_lr, PlotLossesKerasTF()])

        print("Ahoj")

In [13]:
pipeline = DeepLabV3(train_data_dir='data/Feit_colon-annotation_valid/',
                        valid_data_dir='data/Feit_colon-annotation_valid/')

Initializing training datagen

Processing file 1
Processing grid point 663499 out of 663499
Processing file 2
Processing grid point 1599360 out of 1599360
Processing file 3
Processing grid point 1541550 out of 1541550
Processing file 4
Initializing training datagen out of 3518304

Processing file 1
Processing grid point 663499 out of 663499
Processing file 2
Processing grid point 1599360 out of 1599360
Processing file 3
Processing grid point 1541550 out of 1541550
Processing file 4


In [14]:
pipeline.execute_pipeline(perform_validation=True, perform_test_segmentation=False)

Epoch 1/20


2022-05-04 21:40:34.459186: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 325.69MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-05-04 21:40:34.459324: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 297.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2022-05-04 21:40:44.459755: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.00GiB (rounded to 1073741824)requested by op model_2/up_sampling2d_8/resize/ResizeBilinear
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation 

ResourceExhaustedError:  OOM when allocating tensor with shape[16,256,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model_2/up_sampling2d_8/resize/ResizeBilinear
 (defined at /home/jelinek/anaconda3/envs/recetox/lib/python3.8/site-packages/keras/backend.py:3333)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_71934]

Errors may have originated from an input operation.
Input Source operations connected to node model_2/up_sampling2d_8/resize/ResizeBilinear:
In[0] model_2/activation_26/Relu (defined at /home/jelinek/anaconda3/envs/recetox/lib/python3.8/site-packages/keras/backend.py:4867)	
In[1] model_2/up_sampling2d_8/mul (defined at /home/jelinek/anaconda3/envs/recetox/lib/python3.8/site-packages/keras/backend.py:3324)

Operation defined at: (m

In [ ]:
pipeline.save_pipeline()

In [ ]:
eval_model(pipeline.model,
           pipeline.get_data_loader_validation(),
           name,
           print_confusion_matrix=True,
           save_misclassified=True)

In [ ]:
pipeline = FeitDataPipeline.load_pipeline(pipeline_name=name)

In [ ]:
from ml.eval import evaluate_segmentation_on_feit_annotation

evaluation_path = Path('data/Feit_colon-annotation_valid/')

segmentation_dir = Path('segmentations') / pipeline.params.name

evaluate_segmentation_on_feit_annotation(evaluation_path, pipeline.build_segmenter(),
                                         32, pipeline.params.class_names,
                                         save_segmentations=True, segmentations_dir=segmentation_dir,
                                         neighbourhood_size=3)